In [1]:
# Install neccesary packages 

import pandas as pd 
import numpy as np 
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [2]:
# Read file 
data = pd.read_csv("Top_250_Movie.csv")

In [3]:
# Creating a column Views to update the given value 
data["views"] = ""
data.columns

Index(['ID', 'Title', 'Year', 'Rating ', 'Rating_count', 'views'], dtype='object')

In [4]:
# Update the given values to the new columns \

data.loc[data['Title'] == 'Forest Gump', 'views'] = 10000000
data.loc[data['Title'] == 'The Usual Suspects', 'views'] = 7500000
data.loc[data['Title'] == 'Rear Window', 'views'] = 6000000
data.loc[data['Title'] == 'North by Northwest', 'views'] = 4000000
data.loc[data['Title'] == 'The Secret in Their Eyes', 'views'] = 3000000
data.loc[data['Title'] == 'Spotlight', 'views'] = 1000000


In [5]:
data.head()

,ID,Title,Year,Rating,Rating_count,views
0,tt0111161,The Shawshank Redemption,1994,9.2,"20,89,382",
1,tt0068646,The Godfather,1972,9.2,"14,33,884",
2,tt0071562,The Godfather: Part II,1974,9.0,"9,95,997",
3,tt0468569,The Dark Knight,2008,9.0,"20,55,819",
4,tt0050083,12 Angry Men,1957,8.9,"5,91,688",


In [6]:
# Preprocessing the data 

df = data.replace(',', '', regex = True) # Removing the commas to make str object to int
nan_value = float("NaN")
df.replace("", nan_value, inplace=True) # Replacing Nan values to empty cells 

In [7]:
# Create a seperate Dataframe to store values which has Nan
non_nan_df = df[df.isna().any(axis=1)] # Nan dataframe to predict

# Excluding the Nan values 
new = df.dropna() 

In [9]:
new

,ID,Title,Year,Rating,Rating_count,views
28,tt0114814,The Usual Suspects,1995,8.5,903320,7500000.0
45,tt0047396,Rear Window,1954,8.5,397045,6000000.0
76,tt0053125,North by Northwest,1959,8.3,270627,4000000.0
138,tt1305806,The Secret in Their Eyes,2009,8.2,172373,3000000.0
212,tt1895587,Spotlight,2015,8.1,362975,1000000.0


In [10]:
y = new['views']  # Dependant Variable  
x = new.drop(['ID', 'Title', 'Year', 'views'], axis=1) # Independant variables


28     7500000.0
45     6000000.0
76     4000000.0
138    3000000.0
212    1000000.0
Name: views, dtype: float64

In [13]:
# changing type to numeric 
x = x.apply(pd.to_numeric, errors='ignore')

In [14]:
# applying linear regression 
reg = LinearRegression().fit(x, y)

xg_reg = xgb.XGBRegressor(objective ='reg:linear', n_estimators=1)
xg_reg.fit(x,y)

regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x, y)

[17:00:34] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


RandomForestRegressor(max_depth=2, random_state=0)

In [15]:
non_nan_df['ID']

0      tt0111161
1      tt0068646
2      tt0071562
3      tt0468569
4      tt0050083
         ...    
245    tt0070047
246    tt0111495
247    tt0094625
248    tt1954470
249    tt2338151
Name: ID, Length: 245, dtype: object

In [16]:
Id_list = non_nan_df['ID'].to_list()

In [63]:
# df['views'].fillna(0, inplace=True)

In [17]:
# Storing id of movies to estimate it views 


df_pred = pd.DataFrame(columns = ['ID','Title','Year', 'Rating', 'Rating_count', 'views', 'XG_Boost_views','Linear_reg_views','Random_Forest_views'])
for i in Id_list:
    test1 = non_nan_df[(non_nan_df['ID'] == i)]
    test = test1[['Rating ', 'Rating_count']].apply(pd.to_numeric, errors='ignore')
    test1['XG_Boost_views'] = int(xg_reg.predict(test))
    test1['Linear_reg_views'] = int(abs(reg.predict(test)))
    test1['Random_Forest_views'] = int(regr.predict(test))

    df_pred.loc[i] = test1.values[0]


/var/folders/k8/n2zsd8r164l74hwq2bvv9q8m0000gp/T/ipykernel_56690/1611634196.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test1['XG_Boost_views'] = int(xg_reg.predict(test))
/var/folders/k8/n2zsd8r164l74hwq2bvv9q8m0000gp/T/ipykernel_56690/1611634196.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test1['Linear_reg_views'] = int(abs(reg.predict(test)))
/var/folders/k8/n2zsd8r164l74hwq2bvv9q8m0000gp/T/ipykernel_56690/1611634196.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [83]:
# Storing the final estimated value in pred.csv
df_pred.to_csv("Pred.csv") 

In [55]:
df_pred

,ID,Title,Year,Rating,Rating_count,views,XG_Boost_views,Linear_reg_views,Random_Forest_views
tt0111161,tt0111161,The Shawshank Redemption,1994,9.2,2089382,NaN,1230000,17657001,6580000
tt0068646,tt0068646,The Godfather,1972,9.2,1433884,NaN,1230000,16667714,6580000
tt0071562,tt0071562,The Godfather: Part II,1974,9.0,995997,NaN,1230000,13543335,6580000
tt0468569,tt0468569,The Dark Knight,2008,9.0,2055819,NaN,1230000,15142833,6580000
tt0050083,tt0050083,12 Angry Men,1957,8.9,591688,NaN,1230000,11701389,5650000
...,...,...,...,...,...,...,...,...,...
tt0070047,tt0070047,The Exorcist,1973,8.0,332258,NaN,150000,224043,1820000
tt0111495,tt0111495,Three Colors: Red,1994,8.0,79786,NaN,150000,156991,2235000
tt0094625,tt0094625,Akira,1988,8.0,138660,NaN,150000,68137,2235000
tt1954470,tt1954470,Gangs of Wasseypur,2012,8.0,68718,NaN,150000,173695,2235000
